In [2]:
import mlflow
from mlflow.models.signature import infer_signature 
import os

In [ ]:
ENV_FILE = '.env'
EXPERIMENT_NAME = 'mlflow-test'

In [3]:
def get_params(env_file_path):
    with open(os.path.join(env_file_path), 'r') as fh:
        vars_dict = dict(
            tuple(line.replace('\n', '').split('='))
            for line in fh.readlines() if not line.startswith('#')
        )
    return vars_dict

In [4]:
env_values = get_params('.env')
mlflow.set_tracking_uri(f'{env_values.get("MLFLOW_URI")}:{env_values.get("MLFLOW_PORT")}')
mlflow.set_experiment(EXPERIMENT_NAME)

{'MLFLOW_URI': 'http://localhost',
 'MLFLOW_PORT': '5010',
 'DB_PORT': '5455',
 'POSTGRES_USER': 'mlflow',
 'POSTGRES_PASSWORD': 'mlflow',
 'POSTGRES_DB': 'mlflow',
 'S3_BUCKET': 's3://air-pol-mlflow-bucket',
 'S3_ACCESS_KEY_ID': 'jwtnrbj46kqjuds5gi6zbrxcua3a',
 'S3_SECRET_ACCESS_KEY': 'j27itcgzrt6a3g4bgcqfcmfgcximgs3jmfc3x4xtfpxg3lfwm73rk',
 'S3_URI': 'https://gateway.storjshare.io'}

In [ ]:
run_name = 'test_run'
run_id = 0
run = mlflow.start_run()
#mlflow.set_tag("mlflow.runName", run_name)
run_id = mlflow.last_active_run().info.run_id if run_id == 0 else run_id
mlflow.tracking.MlflowClient().set_tag(run_id, "mlflow.runName", run_name)
mlflow.pytorch.autolog()

In [ ]:
model_params = {}
artifact_file = "pipeline/pipeline_params.yaml"
with mlflow.start_run(run_id=run_id):
    mlflow.log_dict(model_params, artifact_file)

In [ ]:
output_onnx_file_path = "pipeline/pipeline.onnx"
artifact_path = "onnx"
with mlflow.start_run(run_id=run_id):
    mlflow.log_artifact(output_onnx_file_path, artifact_path)

In [ ]:
metric_name = 'mse'
train_score = 0
val_score = 0
with mlflow.start_run(run_id=run_id):
    mlflow.log_metric(f'train_{metric_name}_all', train_score)
    mlflow.log_metric(f'val_{metric_name}_all', val_score)

In [ ]:
mlflow.end_run()